In [1]:
# open the file output_processed.txt and read the contents
# print the contents to the screen

# open the file for reading
with open('output-alv.txt', 'r') as file:
    # read the contents of the file
    contents = file.read()
    # print the contents to the screen
    print(contents)
    

[Image: page_1_img_1.jpeg] Annual and Sustainability Report 2023 
--- Page Break ---
 CONTENT 23 Autoliv at a Glance Year in Brief 04 CEO Message – Transforming with a Changing Industry 06 70 Years of Saving More Lives 08 Vision / Mission / Key Behaviors 10 Financial Summary 2023 12 Financial and Sustainability Targets 14 Strategy for Change Customer Focus Strengthening Our Position in a Changing Market 18 Customer Collaboration 20 Competitive Products and Solutions Products for Saving More Lives 22 Innovation as an Enabler to Save More Lives 24 Efficient Value Delivery Operational Excellence 28 Value to Customers 30 The Autoliv Way Outstanding People Embracing Change 32 Sustainability A Driving Force in Sustainable Mobility 34 Sustainability Materiality Assessment 36 Sustainability Governance 38 Road Safety – a Global Challenge 40 Climate and Circularity 43 TCFD Disclosure 48 A Safe and Inclusive Workplace 50 Responsible Business 52 Sustainability Appendix 59 The Autoliv Share Creatin

In [2]:
# Split the text into a list where each element is the content of a page
pages = [x.strip() for x in contents.split('--- Page Break ---')]

In [3]:
pages

['[Image: page_1_img_1.jpeg] Annual and Sustainability Report 2023',
 'CONTENT 23 Autoliv at a Glance Year in Brief 04 CEO Message – Transforming with a Changing Industry 06 70 Years of Saving More Lives 08 Vision / Mission / Key Behaviors 10 Financial Summary 2023 12 Financial and Sustainability Targets 14 Strategy for Change Customer Focus Strengthening Our Position in a Changing Market 18 Customer Collaboration 20 Competitive Products and Solutions Products for Saving More Lives 22 Innovation as an Enabler to Save More Lives 24 Efficient Value Delivery Operational Excellence 28 Value to Customers 30 The Autoliv Way Outstanding People Embracing Change 32 Sustainability A Driving Force in Sustainable Mobility 34 Sustainability Materiality Assessment 36 Sustainability Governance 38 Road Safety – a Global Challenge 40 Climate and Circularity 43 TCFD Disclosure 48 A Safe and Inclusive Workplace 50 Responsible Business 52 Sustainability Appendix 59 The Autoliv Share Creating Shareholder V

In [4]:
import os
from openai import AzureOpenAI

In [5]:
client = AzureOpenAI(
  azure_endpoint = "https://msec-cognitive-sb-eval02-oai.openai.azure.com", 
  api_key="d3116cc2cd9c47fda7474f67aa272385",  
  api_version="2024-02-01",
  timeout=300
)

In [6]:
page_embeddings = []

for page in pages:
    response = client.embeddings.create(model="AA-text-embedding-3-large", input=page)
    embedding = response.data[0].embedding
    page_embeddings.append(embedding)

In [7]:
len(page_embeddings)

172

In [8]:
import numpy as np
import faiss

# Convert the embeddings list to a NumPy array
embeddings_array = np.array(page_embeddings).astype('float32')  # Make sure it's float32

# Dimension of the embeddings
dimension = embeddings_array.shape[1]

# Create a FAISS index
index = faiss.IndexFlatL2(dimension)  # L2 distance metric

# Add the embeddings to the index
index.add(embeddings_array)

# Save the index to disk
faiss.write_index(index, 'faiss_index-alv.index')

print("Index has been saved to disk.")

Index has been saved to disk.


In [11]:
import numpy as np
import faiss

def create_index(client, input_txt_file, out_index_file):
  with open(input_txt_file, 'r') as file:
      # read the contents of the file
      contents = file.read()
      # print the contents to the screen

  
  pages = [x.strip() for x in contents.split('--- Page Break ---')]
  page_embeddings = []

  for page in pages:
      response = client.embeddings.create(model="AA-text-embedding-3-large", input=page)
      embedding = response.data[0].embedding
      page_embeddings.append(embedding)

  # Convert the embeddings list to a NumPy array
  embeddings_array = np.array(page_embeddings).astype('float32')  # Make sure it's float32

  # Dimension of the embeddings
  dimension = embeddings_array.shape[1]

  # Create a FAISS index
  index = faiss.IndexFlatL2(dimension)  # L2 distance metric

  # Add the embeddings to the index
  index.add(embeddings_array)

  # Save the index to disk
  faiss.write_index(index, out_index_file)

  print("Index has been saved to disk.")

In [12]:
create_index(client, 'output_processed.txt', 'faiss_index_attention_is_all_you_need.index')

Index has been saved to disk.
